In [1]:
# === Cell 0 (02以降 共通): レジストリから解決して paths を読む ===
import run_id_registry as runreg
rid = runreg.bootstrap()  # env→ファイル(artifacts/_current/run_id.txt)→Part3→latest→新規 の順で解決

import importlib
import _compat.paths as paths
importlib.reload(paths)

print("[NX] RUN_ID =", rid, "| paths.RUN_ID =", paths.RUN_ID)
paths.ensure_roots()  # artifacts/{RUN_ID}/... を必ず作成
BASE_DIRS = paths.compat_base_dirs  # {'raw','data','models','results','handoff','logs','traces'}


[NX] RUN_ID = 2026-01-10_140940 | paths.RUN_ID = 2026-01-10_140940


# 04-2_statistical_analysis (RUN_ID/lineage対応, joblib優先ローダ)

バージョン: 2025-10-20 rebuild

In [2]:
# === COMPAT BOOTSTRAP (04-2): use _compat/paths.py & _compat/config.json ===
import sys, json
from pathlib import Path

if "_compat" not in sys.path:
    sys.path.append("_compat")

from paths import ensure_roots, compat_base_dirs, load_config as _load_config  # _compat/paths.py
ensure_roots()
RUN_ID = paths.RUN_ID  # [PATCH: unify RUN_ID]

CFG_PATH = Path("_compat/config.json")
if not CFG_PATH.exists():
    raise FileNotFoundError(f"Required config not found: {CFG_PATH}")
cfg = json.loads(CFG_PATH.read_text(encoding="utf-8"))

# output_dirs を compat_base_dirs から構成（既存の output_dirs がある場合は尊重）
if 'output_dirs' not in globals():
    output_dirs = {
        'raw': compat_base_dirs['raw'],
        'data': compat_base_dirs['data'],
        'models': compat_base_dirs['models'],
        'results': compat_base_dirs['results'],
        'handoff': compat_base_dirs['handoff'],
        'logs': compat_base_dirs['logs'],
        'traces': compat_base_dirs['traces'],
    }

print("[COMPAT] RUN_ID =", RUN_ID)
print("[COMPAT] Using cfg from:", CFG_PATH)
print("[COMPAT] handoff dir:", output_dirs['handoff'])

[COMPAT] RUN_ID = 2026-01-10_140940
[COMPAT] Using cfg from: _compat/config.json
[COMPAT] handoff dir: artifacts/2026-01-10_140940/handoff


In [3]:
# === Cell 0 (02以降 共通): レジストリから解決して _compat.paths を読む ===
import importlib, os, json
from pathlib import Path

import run_id_registry as runreg
rid = runreg.bootstrap()  # env→ファイル(artifacts/_current/run_id.txt)→Part3→latest→新規 の順で解決

import _compat.paths as paths
importlib.reload(paths)
importlib.reload(paths)

paths.ensure_roots()
RUN_ID = paths.RUN_ID  # [PATCH: unify RUN_ID]
output_dirs = paths.compat_base_dirs

print("[NX] RUN_ID =", rid, "| paths.RUN_ID =", RUN_ID)
print("[NX] handoff dir:", output_dirs.get("handoff"))

[NX] RUN_ID = 2026-01-10_140940 | paths.RUN_ID = 2026-01-10_140940
[NX] handoff dir: artifacts/2026-01-10_140940/handoff


In [4]:
# === Cell 1: Config loader ===
cfg = {}
cfg_json = Path('_compat/config.json')
if cfg_json.exists():
    with open(cfg_json, 'r', encoding='utf-8') as f:
        cfg = json.load(f)
        print("✅ Loaded _compat/config.json")
else:
    try:
        cfg = paths.load_config()
        print("⚠️ _compat/config.json がないため paths.load_config() を使用")
    except Exception as e:
        cfg = {}
        print("⚠️ load_config() 不在。空設定で継続:", e)

SESSION_ID = cfg.get("session_id") or os.getenv("SESSION_ID") or RUN_ID
cfg.setdefault("run_id", RUN_ID)
cfg.setdefault("session_id", SESSION_ID)
cfg.setdefault("output_dirs", output_dirs)

✅ Loaded _compat/config.json


{}

In [5]:
# === Cell 2: Apply config / paths / DB guard ===
def _coerce_db_config(d: dict) -> dict:
    import os
    d = d or {}
    aliases = {
        "dbname": ["dbname", "database", "db", "name"],
        "user": ["user", "username"],
        "password": ["password", "pass", "pwd"],
        "host": ["host", "hostname"],
        "port": ["port"],
    }
    out = {}
    for k, ks in aliases.items():
        for kk in ks:
            if kk in d and d[kk]:
                out[k] = d[kk]
                break
    # env フォールバック
    env_default = {
        "dbname": os.getenv("PGDATABASE", "rapids_data"),
        "user": os.getenv("PGUSER", "postgres"),
        "password": os.getenv("PGPASSWORD", "postgres"),
        "host": os.getenv("PGHOST", "127.0.0.1"),
        "port": os.getenv("PGPORT", "5432"),
    }
    for k, v in env_default.items():
        out.setdefault(k, v)
    return out

DB_CONFIG = _coerce_db_config(cfg.get("DB_CONFIG", {}) if isinstance(cfg, dict) else {})
if not any(DB_CONFIG.values()):
    print("⚠️ DB_CONFIG が未設定です（04-2では致命的ではないため継続）。")

print("🔧 DB_CONFIG ready:", {k: DB_CONFIG.get(k, "-") for k in ["dbname","host","port","user"]})
print("cfg keys:", sorted(list(cfg.keys()))[:20], "...")

🔧 DB_CONFIG ready: {'dbname': 'rapids_data', 'host': 'localhost', 'port': '5432', 'user': 'postgres'}
cfg keys: ['DB_CONFIG', 'analysis', 'brand_keywords', 'db', 'engine', 'free_ca_list', 'full_processing', 'handoff', 'llm', 'model', 'output_dirs', 'paths', 'run_id', 'session_id', 'system', 'tld_analysis', 'visualization'] ...


In [6]:
# === Cell 2.3: handoff resolver（入力: 04-1 の成果を読む） ===
import glob, time, pickle
from datetime import datetime

def _read_json(path: Path):
    try:
        if path.exists():
            with open(path, 'r', encoding='utf-8') as f:
                return json.load(f)
    except Exception as e:
        print(f"⚠️ JSON read failed: {path} | {e}")
    return None

def _same_run_id(p: Path) -> bool:
    try:
        return str(p).split("artifacts/")[1].split("/")[0] == RUN_ID
    except Exception:
        return False

def _joblib_first_load(p: Path):
    try:
        import joblib
        return joblib.load(p)
    except Exception as e1:
        try:
            with open(p, 'rb') as f:
                return pickle.load(f)
        except Exception as e2:
            raise RuntimeError(f"Failed to load handoff via joblib and pickle: {e1} | {e2}")

def resolve_handoff_04_1():
    # (1) ENV
    env_p = os.getenv("HANDOFF_IN_PATH")
    if env_p:
        ep = Path(env_p)
        if ep.exists() and _same_run_id(ep):
            return ep, "env"

    # (2) cfg.handoff["04-1"]["path"]
    try:
        cfg_p = cfg.get("handoff", {}).get("04-1", {}).get("path")
        if cfg_p:
            cp = Path(cfg_p)
            if cp.exists() and _same_run_id(cp):
                return cp, "cfg"
    except Exception:
        pass

    # (3) pointers
    for lp in [Path("artifacts/_current_lineage.json"),
               Path(output_dirs["handoff"]) / "_lineage.json"]:
        j = _read_json(lp)
        if not j:
            continue
        items = []
        if isinstance(j, list):
            items = j
        elif isinstance(j, dict):
            items = j.get("items", [])
            last = j.get("last")
            if isinstance(last, dict) and str(last.get("type","")).startswith("04-1"):
                p = Path(last.get("path",""))
                if p.exists() and _same_run_id(p):
                    return p, "pointer(last)"
        for it in reversed(items):
            try:
                if str(it.get("type","")).startswith("04-1"):
                    p = Path(it.get("path",""))
                    if p.exists() and _same_run_id(p):
                        return p, "pointer(items)"
            except Exception:
                continue

    # (4) default strict name
    strict = Path(output_dirs["handoff"]) / "04-1_config_and_data_preparation.pkl"
    if strict.exists() and _same_run_id(strict):
        return strict, "strict"

    # (5) wildcard newest
    for pat in [str(Path(output_dirs["handoff"]) / "04-1_*.pkl"),
                str(Path(output_dirs["handoff"]) / "04-1*.pkl")]:
        files = sorted(glob.glob(pat), key=lambda p: Path(p).stat().st_mtime, reverse=True)
        for f in files:
            fp = Path(f)
            if fp.exists() and _same_run_id(fp):
                return fp, "wildcard"

    all04 = sorted(Path(output_dirs["handoff"]).glob("04-1*.pkl"))
    same_run = [str(p) for p in all04 if _same_run_id(p)]
    raise FileNotFoundError(f"[resolver] 04-1 handoff not found under same RUN_ID. candidates(same-run)={same_run}")

handoff_in_path, handoff_source = None, None
try:
    handoff_in_path, handoff_source = resolve_handoff_04_1()
    print(f"[resolver] handoff_in={handoff_in_path} (source={handoff_source})")
    handoff_in = _joblib_first_load(handoff_in_path)
except Exception as e:
    print("⚠️ resolver/load failed:", e)
    handoff_in = {}

for k in ["cfg", "RUN_ID", "output_dirs"]:
    if not isinstance(handoff_in, dict) or k not in handoff_in:
        print(f"⚠️ [handoff] missing key: {k}")

import pandas as pd
false_negatives_df = handoff_in.get("false_negatives_df", pd.DataFrame())
brand_keywords = handoff_in.get("brand_keywords", [])
cert_full_info_map = handoff_in.get("cert_full_info_map", {})
print(f"[handoff] false_negatives_df={len(false_negatives_df)} rows | brand_keywords={len(brand_keywords)} | cert_map={len(cert_full_info_map)}")

[resolver] handoff_in=artifacts/2026-01-10_140940/handoff/04-1_config_and_data_preparation.pkl (source=strict)
[handoff] false_negatives_df=55524 rows | brand_keywords=109 | cert_map=55524


In [7]:
# === analysis stats resolver (auto-added) ===
# 目的: HIGH_RISK_WORDS, suspicious_words_stats, TLD_STATS, KNOWN_DOMAINS を可能な範囲で初期化
import pandas as pd

# HIGH_RISK_WORDS: brand_keywords を優先
if 'HIGH_RISK_WORDS' not in globals():
    if 'brand_keywords' in globals() and isinstance(brand_keywords, (list, set, dict)):
        if isinstance(brand_keywords, dict):
            HIGH_RISK_WORDS = sorted(set(sum(([k] + (v if isinstance(v, list) else []) for k,v in brand_keywords.items()), [])))
        else:
            HIGH_RISK_WORDS = sorted(list(set(brand_keywords)))
        print(f"🔧 HIGH_RISK_WORDS derived from brand_keywords (n={len(HIGH_RISK_WORDS)})")
    else:
        HIGH_RISK_WORDS = []
        print("⚠️ HIGH_RISK_WORDS fallback: empty list")

# suspicious_words_stats: false_negatives_df があれば簡易頻度集計（トークナイズは最小限）
if 'suspicious_words_stats' not in globals():
    try:
        if 'false_negatives_df' in globals() and hasattr(false_negatives_df, 'assign'):
            df = false_negatives_df.copy()
            # 最小限：ドメインの '-' '_' '.' で分割して単語頻度
            def split_words(s):
                try:
                    for ch in ['-', '_', '.']:
                        s = s.replace(ch, ' ')
                    return [w for w in s.split() if w]
                except Exception:
                    return []
            words = df['domain'].astype(str).map(split_words)
            from collections import Counter
            cnt = Counter([w for lst in words for w in lst])
            suspicious_words_stats = pd.DataFrame(cnt.items(), columns=['word','count']).sort_values('count', ascending=False)
            print(f"🔧 suspicious_words_stats derived (unique={len(suspicious_words_stats)})")
        else:
            suspicious_words_stats = pd.DataFrame(columns=['word','count'])
            print("⚠️ suspicious_words_stats fallback: empty frame")
    except Exception as e:
        suspicious_words_stats = pd.DataFrame(columns=['word','count'])
        print("⚠️ suspicious_words_stats error -> empty:", e)

# TLD_STATS: false_negatives_df があれば簡易集計
if 'TLD_STATS' not in globals():
    try:
        if 'false_negatives_df' in globals() and 'domain' in false_negatives_df.columns:
            def get_tld(d):
                try:
                    parts = str(d).split('.')
                    return parts[-1] if len(parts)>1 else ''
                except Exception:
                    return ''
            tlds = false_negatives_df['domain'].map(get_tld)
            TLD_STATS = tlds.value_counts().rename_axis('tld').reset_index(name='count')
            print(f"🔧 TLD_STATS derived (unique={len(TLD_STATS)})")
        else:
            import pandas as pd
            TLD_STATS = pd.DataFrame(columns=['tld','count'])
            print("⚠️ TLD_STATS fallback: empty frame")
    except Exception as e:
        import pandas as pd
        TLD_STATS = pd.DataFrame(columns=['tld','count'])
        print("⚠️ TLD_STATS error -> empty:", e)

# KNOWN_DOMAINS: false_negatives_df or cert_full_info_map から推定
if 'KNOWN_DOMAINS' not in globals():
    try:
        if 'false_negatives_df' in globals() and 'domain' in false_negatives_df.columns:
            KNOWN_DOMAINS = set(false_negatives_df['domain'].dropna().astype(str).tolist())
            print(f"🔧 KNOWN_DOMAINS derived from false_negatives_df (n={len(KNOWN_DOMAINS)})")
        elif 'cert_full_info_map' in globals() and isinstance(cert_full_info_map, dict):
            KNOWN_DOMAINS = set(cert_full_info_map.keys())
            print(f"🔧 KNOWN_DOMAINS derived from cert_full_info_map (n={len(KNOWN_DOMAINS)})")
        else:
            KNOWN_DOMAINS = set()
            print("⚠️ KNOWN_DOMAINS fallback: empty set")
    except Exception as e:
        KNOWN_DOMAINS = set()
        print("⚠️ KNOWN_DOMAINS error -> empty:", e)

print("✅ stats resolver completed.")

🔧 HIGH_RISK_WORDS derived from brand_keywords (n=109)
🔧 suspicious_words_stats derived (unique=55892)
🔧 TLD_STATS derived (unique=504)
🔧 KNOWN_DOMAINS derived from false_negatives_df (n=55524)
✅ stats resolver completed.


In [8]:
# === stats normalize (auto-added) ===
# 目的: 下流セルが dict.keys() 前提でも落ちないよう、型を正規化
def _to_domain_dict(obj):
    if obj is None:
        return {}
    if isinstance(obj, dict):
        # すでに dict の場合はキーを str に寄せる
        return {str(k): v for k, v in obj.items()}
    if isinstance(obj, (set, list, tuple)):
        return {str(x): True for x in obj}
    # 想定外は空にフォールバック
    return {}

# KNOWN_DOMAINS を dict に正規化（.keys() を安全に呼べるようにする）
if 'KNOWN_DOMAINS' in globals():
    KNOWN_DOMAINS = _to_domain_dict(KNOWN_DOMAINS)

# HIGH_RISK_WORDS は list へ寄せて重複排除
if 'HIGH_RISK_WORDS' in globals():
    if isinstance(HIGH_RISK_WORDS, dict):
        HIGH_RISK_WORDS = list(HIGH_RISK_WORDS.keys())
    elif isinstance(HIGH_RISK_WORDS, (set, tuple)):
        HIGH_RISK_WORDS = list(HIGH_RISK_WORDS)
    HIGH_RISK_WORDS = sorted(list(set(map(str, HIGH_RISK_WORDS))))

print("✅ stats normalized:",
      "KNOWN_DOMAINS=dict" if isinstance(globals().get('KNOWN_DOMAINS', None), dict) else type(globals().get('KNOWN_DOMAINS', None)),
      f"HIGH_RISK_WORDS={len(globals().get('HIGH_RISK_WORDS', []))}")

✅ stats normalized: KNOWN_DOMAINS=dict HIGH_RISK_WORDS=109


In [9]:
# === brand_keywords resolver (auto-added) ===
# 優先: handoff -> artifacts/{RUN_ID}/models/brand_keywords.json -> models/*/brand_keywords.json
import os, json
from pathlib import Path

def _load_brand_keywords_candidates():
    cands = []
    # 1) handoff
    if 'handoff' in globals() and isinstance(handoff, dict) and 'brand_keywords' in handoff:
        return handoff['brand_keywords']
    # 2) artifacts/{RUN_ID}/models
    try:
        p = Path(MODELS_DIR) / "brand_keywords.json"
        if p.exists():
            with p.open("r", encoding="utf-8") as f:
                return json.load(f)
    except Exception:
        pass
    # 3) legacy models/*/brand_keywords.json (latest)
    try:
        latest = None
        models_root = Path("models")
        if models_root.exists():
            dirs = sorted([d for d in models_root.glob("*") if d.is_dir()])
            if dirs:
                latest = dirs[-1]
                p = latest / "brand_keywords.json"
                if p.exists():
                    with p.open("r", encoding="utf-8") as f:
                        return json.load(f)
    except Exception:
        pass
    return None

if 'brand_keywords' not in globals() or brand_keywords in (None, [], {}):
    _bk = _load_brand_keywords_candidates()
    if _bk is not None:
        brand_keywords = _bk
        print("🔧 brand_keywords loaded")
    else:
        print("⚠️ brand_keywords not found in handoff nor models.")

In [10]:
# === DB_CONFIG guard (mirrors 01_data_preparation, refined) ===
import os, json, warnings
from pathlib import Path
from typing import Optional

# --- DB_CONFIG (same behavior) ---
if 'DB_CONFIG' not in globals():
    if 'handoff' in globals() and isinstance(handoff, dict) and 'DB_CONFIG' in handoff:
        DB_CONFIG = handoff['DB_CONFIG']
    else:
        DB_CONFIG = {
            'dbname': os.getenv('PGDATABASE','rapids_data'),
            'user': os.getenv('PGUSER','postgres'),
            'password': os.getenv('PGPASSWORD','asomura'),
            'host': os.getenv('PGHOST','localhost'),
            'port': os.getenv('PGPORT','5432'),
        }
print("🔧 DB_CONFIG ready:", {k: DB_CONFIG[k] for k in ['dbname','host','port','user']})

# --- handoff unpack (only for keys that should exist as inputs) ---
optional_from_04_1 = [
    'false_negatives_df',
    'fn_features_df',
    'brand_keywords',
    'cert_full_info_map',
]
generated_in_04_2 = [
    'DANGEROUS_TLDS',
    'LEGITIMATE_TLDS',
    'NEUTRAL_TLDS',
]

# Unpack what is present (inputs)
if 'handoff' in globals() and isinstance(handoff, dict):
    for k in optional_from_04_1:
        if k in handoff and globals().get(k) is None:
            globals()[k] = handoff[k]

missing_input = [k for k in optional_from_04_1 if globals().get(k) is None]
if missing_input:
    print(f"ℹ️ optional inputs missing at load-time: {missing_input} — trying to backfill (lineage → 04-1 → cfg).")
    # --- Backfill from lineage → 04-1 handoff ---
    def _resolve_04_1_from_lineage() -> Optional[Path]:
        ptrs = [Path('artifacts/_current_lineage.json'), Path(compat_base_dirs['handoff']) / '_lineage.json']
        for ptr in ptrs:
            try:
                if ptr.exists():
                    data = json.loads(ptr.read_text(encoding='utf-8'))
                    # Handle common shapes: items[-1] or last
                    candidates = []
                    if isinstance(data, dict):
                        if 'last' in data and isinstance(data['last'], dict) and 'path' in data['last']:
                            candidates.append(Path(data['last']['path']).parent / '04-1_config_and_data_preparation.pkl')
                        if 'items' in data:
                            for it in data['items']:
                                if isinstance(it, dict) and it.get('type') == '04-1_handoff' and 'path' in it:
                                    candidates.append(Path(it['path']))
                    for cp in candidates:
                        if cp.exists():
                            return cp
            except Exception as e:
                print(f"[Auto] lineage scan error: {e}")
        return None

    h1 = _resolve_04_1_from_lineage()
    if h1 and h1.exists():
        try:
            import pickle
            with open(h1, 'rb') as f:
                h1_payload = pickle.load(f)
            for k in missing_input:
                if globals().get(k) is None and k in h1_payload:
                    globals()[k] = h1_payload[k]
            missing_input = [k for k in optional_from_04_1 if globals().get(k) is None]
            if missing_input:
                print(f"ℹ️ 04-1 handoff でも見つからない: {missing_input}")
        except Exception as e:
            print(f"⚠️ 04-1 handoff load failed: {e}")

    # --- Backfill from cfg if available ---
    if globals().get('cfg'):
        if 'brand_keywords' in missing_input and 'brand_keywords' in cfg:
            globals()['brand_keywords'] = cfg['brand_keywords']
        if 'cert_full_info_map' in missing_input and 'cert_full_info_map' in cfg:
            globals()['cert_full_info_map'] = cfg['cert_full_info_map']

    missing_input = [k for k in optional_from_04_1 if globals().get(k) is None]
    if missing_input:
        warnings.warn(f"optional inputs still missing after backfill: {missing_input} — pipeline will continue with safe defaults.")

print("ℹ️ keys to be produced in this module:", generated_in_04_2)
print("✅ handoff unpack (partial allowed)")

🔧 DB_CONFIG ready: {'dbname': 'rapids_data', 'host': 'localhost', 'port': '5432', 'user': 'postgres'}
ℹ️ optional inputs missing at load-time: ['fn_features_df'] — trying to backfill (lineage → 04-1 → cfg).
ℹ️ keys to be produced in this module: ['DANGEROUS_TLDS', 'LEGITIMATE_TLDS', 'NEUTRAL_TLDS']
✅ handoff unpack (partial allowed)


In [11]:
"""
プログラム名: 03_ai_agent_analysis.ipynb
概要: MLモデルが見逃したフィッシングサイトをAIエージェント（Qwen3-14B）で検出
目的: 国際学会論文での発表（MLとLLMのハイブリッドアプローチの実証）
作成日: 2025-01-17
"""

"""
セル番号: 04-IMPROVED
概要: 分析結果を活かした改良版判定ロジック
入力: HIGH_RISK_WORDS, TLD_STATS, KNOWN_DOMAINS（セル04の実行結果）
出力: 改良版の判定関数群
"""

import re
from typing import Dict, List, Tuple, Optional
import math

print("=" * 80)
print("🚀 分析結果を活かした改良版判定システム")
print("=" * 80)

# ========== 1. 分析結果の確認 ==========

print("\n📊 利用可能な統計データ:")
print("-" * 40)

# 必要な変数の確認
required_vars = {
    'HIGH_RISK_WORDS': '高リスク単語リスト',
    'suspicious_words_stats': '単語別統計',
    'TLD_STATS': 'TLD別統計',
    'KNOWN_DOMAINS': '埋め込みドメイン統計'
}

available_vars = {}
for var_name, description in required_vars.items():
    if var_name in globals():
        available_vars[var_name] = globals()[var_name]
        if isinstance(globals()[var_name], list):
            print(f"✅ {var_name}: {len(globals()[var_name])}個")
        elif isinstance(globals()[var_name], dict):
            print(f"✅ {var_name}: {len(globals()[var_name])}種類")
    else:
        print(f"❌ {var_name}: 未定義 - {description}")

if len(available_vars) < 3:
    print("\n⚠️ セル04を先に実行してください")

# ========== 2. 改良版の特徴抽出関数 ==========

def extract_features_improved(domain: str) -> Dict:
    """
    分析結果を活かした改良版特徴抽出
    
    Returns:
        {
            'clean_words': [],      # 通常の単語（問題のあるものを除外）
            'risk_patterns': [],    # 検出されたリスクパターン
            'risk_scores': {},      # パターン別のリスクスコア
            'tld_info': {},        # TLD情報
            'structure': {}        # 構造情報
        }
    """
    
    features = {
        'clean_words': [],
        'risk_patterns': [],
        'risk_scores': {},
        'tld_info': {},
        'structure': {}
    }
    
    if not domain:
        return features
    
    domain_lower = domain.lower().strip()
    
    # ========== IPアドレスチェック ==========
    # 数字のTLDを防ぐ
    if re.match(r'^[\d\.]+$', domain_lower):
        features['risk_patterns'].append('ip_address')
        features['risk_scores']['ip_address'] = 30
        return features
    
    # ========== 前処理 ==========
    # www.を除去（59,046件の誤検出を防ぐ）
    if domain_lower.startswith('www.'):
        domain_lower = domain_lower[4:]
        features['structure']['has_www'] = True
    
    # ========== TLD抽出（改良版） ==========
    if 'extract_real_tld' in globals():
        tld_info = extract_real_tld(domain_lower)
        features['tld_info'] = tld_info
        domain_body = tld_info.get('domain_body', domain_lower)
    else:
        # フォールバック
        parts = domain_lower.split('.')
        if len(parts) >= 2:
            domain_body = '.'.join(parts[:-1])
            features['tld_info'] = {'tld': parts[-1], 'domain_body': domain_body}
        else:
            domain_body = domain_lower
            features['tld_info'] = {'tld': '', 'domain_body': domain_body}
    
    # ========== 構造分析 ==========
    features['structure']['depth'] = domain_lower.count('.') + 1
    features['structure']['length'] = len(domain_lower)
    features['structure']['hyphen_count'] = domain_lower.count('-')
    
    # ========== リスクパターン検出（優先度順） ==========
    
    # パターン1: 確実な偽装（最優先）
    if '.co.jp.' in domain_lower and not domain_lower.endswith('.co.jp'):
        features['risk_patterns'].append('fake_cojp_absolute')
        features['risk_scores']['fake_cojp_absolute'] = 95
    
    if '.com.' in domain_lower and not domain_lower.endswith('.com'):
        features['risk_patterns'].append('fake_com_absolute')
        features['risk_scores']['fake_com_absolute'] = 90
    
    # パターン2: 既知ドメインの埋め込み（高優先）
    if 'KNOWN_DOMAINS' in globals():
        for known_domain in KNOWN_DOMAINS.keys():
            if known_domain in domain_lower and not domain_lower.endswith(known_domain):
                pattern_name = f'embedded_{known_domain.replace(".", "_")}'
                features['risk_patterns'].append(pattern_name)
                # 埋め込み回数に基づくスコア（KNOWN_DOMAINS の型揺れに耐える）
                # CHANGELOG: 2025-12-28 bool is not subscriptable 対策（stats dict 以外は 0扱い）
                embed_stats = KNOWN_DOMAINS.get(known_domain, {})
                phish_embedded = embed_stats.get('phishing_embedded', 0) if isinstance(embed_stats, dict) else 0
                if phish_embedded > 1000:
                    features['risk_scores'][pattern_name] = 90
                elif phish_embedded > 500:
                    features['risk_scores'][pattern_name] = 85
                elif phish_embedded > 100:
                    features['risk_scores'][pattern_name] = 80
                else:
                    features['risk_scores'][pattern_name] = 70
    
    # パターン3: ハイフン偽装パターン
    jp_hyphen_patterns = ['-jp.', '.jp-', '-jp-', 'jp-']
    co_hyphen_patterns = ['-co.', '.co-', '-co-', 'co-']
    
    for pattern in jp_hyphen_patterns:
        if pattern in domain_body:
            features['risk_patterns'].append('jp_hyphen_variant')
            features['risk_scores']['jp_hyphen_variant'] = 60
            break
    
    for pattern in co_hyphen_patterns:
        if pattern in domain_body:
            features['risk_patterns'].append('co_hyphen_variant')
            features['risk_scores']['co_hyphen_variant'] = 55
            break
    
    # パターン4: DuckDNS（最危険）
    if 'duckdns' in domain_lower:
        features['risk_patterns'].append('duckdns')
        features['risk_scores']['duckdns'] = 100
    
    # ========== 単語抽出（改良版） ==========
    words_raw = re.findall(r'[a-z]+', domain_body)
    
    # 条件付きフィルタリング
    for word in words_raw:
        # 単独の'jp'や'co'の扱い
        if word == 'jp':
            # 偽装パターンがある場合のみリスクとして扱う
            if any(p in features['risk_patterns'] for p in 
                   ['fake_cojp_absolute', 'jp_hyphen_variant']):
                features['risk_patterns'].append('jp_in_suspicious_context')
                features['risk_scores']['jp_in_suspicious_context'] = 40
            # それ以外は無視（正常な使用の可能性）
            
        elif word == 'co':
            # 偽装パターンがある場合のみリスクとして扱う
            if any(p in features['risk_patterns'] for p in 
                   ['fake_cojp_absolute', 'co_hyphen_variant']):
                features['risk_patterns'].append('co_in_suspicious_context')
                features['risk_scores']['co_in_suspicious_context'] = 35
            # それ以外は無視
            
        elif word == 'www':
            # wwwは完全に無視（既に除去済み）
            pass
            
        else:
            # 通常の単語
            if 2 <= len(word) <= 15:
                features['clean_words'].append(word)
    
    return features

# ========== 3. リスク評価関数（改良版） ==========

def calculate_risk_score_improved(domain: str) -> Dict:
    """
    分析結果を活かした総合リスク評価
    """
    
    features = extract_features_improved(domain)
    
    result = {
        'domain': domain,
        'total_risk': 0,
        'confidence': 0,
        'risk_breakdown': {},
        'verdict': 'unknown',
        'explanation': []
    }
    
    # ========== パターンベースのリスク（最優先） ==========
    pattern_risk = 0
    if features['risk_patterns']:
        # 最も高いリスクスコアを採用
        pattern_risk = max(features['risk_scores'].values())
        result['risk_breakdown']['pattern'] = pattern_risk
        
        # 説明を追加
        for pattern in features['risk_patterns']:
            if pattern == 'fake_cojp_absolute':
                result['explanation'].append("確実な.co.jp偽装パターン")
            elif pattern.startswith('embedded_'):
                domain_name = pattern.replace('embedded_', '').replace('_', '.')
                result['explanation'].append(f"{domain_name}の埋め込み偽装")
            elif pattern == 'duckdns':
                result['explanation'].append("DuckDNS（フリーDNS）使用")
    
    # ========== TLDベースのリスク ==========
    tld_risk = 0
    if 'TLD_STATS' in globals() and features['tld_info'].get('tld'):
        tld = features['tld_info']['tld']
        if tld in TLD_STATS:
            stats = TLD_STATS[tld]
            odds_ratio = stats['odds_ratio']
            
            if odds_ratio == float('inf') or odds_ratio > 100:
                tld_risk = 40
                result['explanation'].append(f"危険TLD: .{tld}")
            elif odds_ratio > 10:
                tld_risk = 25
                result['explanation'].append(f"要注意TLD: .{tld}")
            elif odds_ratio < 0.1:
                tld_risk = -20  # 安全なTLDはリスクを下げる
                result['explanation'].append(f"信頼TLD: .{tld}")
    
    result['risk_breakdown']['tld'] = tld_risk
    
    # ========== 単語ベースのリスク（補助的） ==========
    word_risk = 0
    if 'suspicious_words_stats' in globals() and features['clean_words']:
        risk_words = []
        for word in features['clean_words']:
            if word in suspicious_words_stats:
                stats = suspicious_words_stats[word]
                if stats['odds_ratio'] > 50:
                    risk_words.append(word)
                    word_risk += 5
        
        if risk_words:
            word_risk = min(word_risk, 30)  # 最大30点
            result['risk_breakdown']['words'] = word_risk
            result['explanation'].append(f"危険単語: {', '.join(risk_words[:3])}")
    
    # ========== 構造的リスク ==========
    structure_risk = 0
    if features['structure']['depth'] > 4:
        structure_risk += 15
        result['explanation'].append(f"階層が深い: {features['structure']['depth']}層")
    
    if features['structure']['length'] > 50:
        structure_risk += 10
        result['explanation'].append(f"異常に長い: {features['structure']['length']}文字")
    
    result['risk_breakdown']['structure'] = structure_risk
    
    # ========== 総合評価 ==========
    # パターンリスクを重視
    if pattern_risk >= 80:
        # 確実な偽装パターン
        result['total_risk'] = pattern_risk
        result['confidence'] = 0.95
    elif pattern_risk >= 60:
        # 高リスクパターン + 他の要因
        result['total_risk'] = pattern_risk + (tld_risk + word_risk + structure_risk) * 0.3
        result['confidence'] = 0.80
    else:
        # 通常の重み付け
        result['total_risk'] = pattern_risk * 0.5 + tld_risk * 0.3 + word_risk * 0.15 + structure_risk * 0.05
        result['confidence'] = min(0.7, result['total_risk'] / 100)
    
    # 正規化
    result['total_risk'] = min(100, max(0, result['total_risk']))
    
    # 判定
    if result['total_risk'] >= 60:
        result['verdict'] = 'phishing'
    elif result['total_risk'] <= 30:
        result['verdict'] = 'legitimate'
    else:
        result['verdict'] = 'suspicious'
    
    return result

# ========== 4. テスト実行 ==========

print("\n🧪 改良版判定システムのテスト:")
print("-" * 40)

test_cases = [
    # 確実なフィッシング
    'amazon.co.jp.suppor34.frhnkm.ph',
    'amazon-co-jp.com',
    'rakuten-jp.net',
    
    # 正規サイト
    'kobe-denki.co.jp',
    'amazon.co.jp',
    'google.com',
    
    # グレーゾーン
    'jp-bank.com',
    'secure-payment.tk',
    'duckdns.org'
]


# ========== Smoke test (optional) ==========
# CHANGELOG: 2025-12-28 04-2 の生成物に必須ではないため、デフォルトでは実行しない
import os as _os
_run_smoke = str(_os.getenv("RUN_SMOKE_TESTS", "0")).lower() in ("1", "true", "yes") or bool(globals().get("DEVELOPMENT_MODE", False))
if not _run_smoke:
    print("ℹ️ Smoke tests skipped. Set RUN_SMOKE_TESTS=1 to run.")
else:
    for domain in test_cases:
        result = calculate_risk_score_improved(domain)
    
        # 判定マーク
        if result['verdict'] == 'phishing':
            mark = '🚨'
        elif result['verdict'] == 'legitimate':
            mark = '✅'
        else:
            mark = '⚠️'
    
        print(f"\n{domain}")
        print(f"  {mark} 判定: {result['verdict']} (リスク: {result['total_risk']:.1f}, 信頼度: {result['confidence']:.2f})")
        if result['explanation']:
            print(f"  理由: {'; '.join(result['explanation'][:3])}")
# ========== 5. 統計サマリー ==========

if 'HIGH_RISK_WORDS' in globals():
    # 問題のある単語の除外確認
    problematic_words = ['jp', 'co', 'www']
    clean_high_risk = [w for w in HIGH_RISK_WORDS if w not in problematic_words]
    
    print(f"\n📊 改良の効果:")
    print(f"  - 元の高リスク単語: {len(HIGH_RISK_WORDS)}個")
    print(f"  - 問題のある単語を除外: {len(HIGH_RISK_WORDS) - len(clean_high_risk)}個")
    print(f"  - パターン検出を優先: 偽装パターンは95点以上")
    print(f"  - TLD統計を活用: {len(TLD_STATS) if 'TLD_STATS' in globals() else 0}種類")

# ========== 6. グローバル変数として保存 ==========

globals()['extract_features_improved'] = extract_features_improved
globals()['calculate_risk_score_improved'] = calculate_risk_score_improved

print("\n" + "=" * 80)
print("✅ 改良版判定システム準備完了")
print("=" * 80)

print("\n💡 改良のポイント:")
print("  1. 偽装パターン（10,695件）を最優先で検出")
print("  2. 埋め込み偽装（1,190件）を確実に検出")
print("  3. 'jp'や'co'は文脈を見て判断")
print("  4. wwwを完全に除外（59,046件の誤検出を防ぐ）")
print("  5. IPアドレスを正しく認識")
print("  6. TLD統計（810種類）を活用")

🚀 分析結果を活かした改良版判定システム

📊 利用可能な統計データ:
----------------------------------------
✅ HIGH_RISK_WORDS: 109個
✅ KNOWN_DOMAINS: 55524種類

🧪 改良版判定システムのテスト:
----------------------------------------
ℹ️ Smoke tests skipped. Set RUN_SMOKE_TESTS=1 to run.

📊 改良の効果:
  - 元の高リスク単語: 109個
  - 問題のある単語を除外: 0個
  - パターン検出を優先: 偽装パターンは95点以上
  - TLD統計を活用: 504種類

✅ 改良版判定システム準備完了

💡 改良のポイント:
  1. 偽装パターン（10,695件）を最優先で検出
  2. 埋め込み偽装（1,190件）を確実に検出
  3. 'jp'や'co'は文脈を見て判断
  4. wwwを完全に除外（59,046件の誤検出を防ぐ）
  5. IPアドレスを正しく認識
  6. TLD統計（810種類）を活用


In [12]:
# === Cell 4: handoff 保存 & ポインタ更新（出力: 04-2） ===
from pathlib import Path
import cloudpickle
from datetime import datetime
import json

handoff_out = Path(output_dirs['handoff']) / '04-2_statistical_analysis.pkl'

# pass-through (from 04-1) があれば確実に引き継ぐ
_pt_brand_keywords = globals().get("brand_keywords", [])
_pt_cert_map = globals().get("cert_full_info_map", {})
_pt_fn_df = globals().get("false_negatives_df", None)
_pt_features_df = globals().get("fn_features_df", None)  # あれば同梱

handoff_04_2 = {
    # --- meta ---
    "cfg": cfg,
    "RUN_ID": RUN_ID,
    "SESSION_ID": SESSION_ID,
    "output_dirs": output_dirs,
    # --- pass-through from 04-1 ---
    "brand_keywords": _pt_brand_keywords,
    "cert_full_info_map": _pt_cert_map,
    "false_negatives_df": _pt_fn_df,
    "fn_features_df": _pt_features_df,
    # --- 04-2 artifacts ---
    "HIGH_RISK_WORDS": globals().get("HIGH_RISK_WORDS"),
    "suspicious_words_stats": globals().get("suspicious_words_stats"),
    "TLD_STATS": globals().get("TLD_STATS"),
    "KNOWN_DOMAINS": globals().get("KNOWN_DOMAINS"),
    "DANGEROUS_TLDS": globals().get("DANGEROUS_TLDS"),
    "LEGITIMATE_TLDS": globals().get("LEGITIMATE_TLDS"),
    "NEUTRAL_TLDS": globals().get("NEUTRAL_TLDS"),
    "extract_features_improved": globals().get("extract_features_improved"),
    "calculate_risk_score_improved": globals().get("calculate_risk_score_improved"),
}

handoff_out.parent.mkdir(parents=True, exist_ok=True)
with open(handoff_out, 'wb') as f:
    cloudpickle.dump(handoff_04_2, f)

print(f"✅ Saved: {handoff_out}")

def _append_lineage(json_path: Path, entry: dict):
    data = None
    try:
        if json_path.exists():
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
    except Exception:
        data = None

    if isinstance(data, list):
        data.append(entry)
    elif isinstance(data, dict):
        items = data.get("items", [])
        if not isinstance(items, list):
            items = []
        items.append(entry)
        data["items"] = items
        data["last"] = {"type": entry.get("type"), "path": entry.get("path")}
    else:
        data = {"items": [entry], "last": {"type": entry.get("type"), "path": entry.get("path")}}

    json_path.parent.mkdir(parents=True, exist_ok=True)
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

entry = {
    "type": "04-2_handoff",
    "path": str(handoff_out),
    "RUN_ID": RUN_ID,
    "updated_at": datetime.utcnow().isoformat() + "Z",
}
_append_lineage(Path("artifacts/_current_lineage.json"), entry)
_append_lineage(Path(output_dirs['handoff']) / "_lineage.json", entry)
print("✅ Pointers updated.")

✅ Saved: artifacts/2026-01-10_140940/handoff/04-2_statistical_analysis.pkl
✅ Pointers updated.


/tmp/ipykernel_1197400/1680315291.py:73: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "updated_at": datetime.utcnow().isoformat() + "Z",


In [13]:
# === Cell 5: Summary ===
from pathlib import Path
print("RUN_ID:", RUN_ID)
print("SESSION_ID:", SESSION_ID)
print("handoff_in source:", globals().get("handoff_source"))
print("handoff_in path:", globals().get("handoff_in_path"))
print("handoff_out:", str(Path(output_dirs['handoff']) / '04-2_statistical_analysis.pkl'))
print("cfg keys (head):", sorted(list(cfg.keys()))[:20])
missing = [k for k in ["HIGH_RISK_WORDS","TLD_STATS","extract_features_improved","calculate_risk_score_improved"] if globals().get(k) is None]
if missing:
    print("⚠️ Missing generated keys:", missing)
else:
    print("✅ Required artifacts present.")

RUN_ID: 2026-01-10_140940
SESSION_ID: 2026-01-10_140940
handoff_in source: strict
handoff_in path: artifacts/2026-01-10_140940/handoff/04-1_config_and_data_preparation.pkl
handoff_out: artifacts/2026-01-10_140940/handoff/04-2_statistical_analysis.pkl
cfg keys (head): ['DB_CONFIG', 'analysis', 'brand_keywords', 'db', 'engine', 'free_ca_list', 'full_processing', 'handoff', 'llm', 'model', 'output_dirs', 'paths', 'run_id', 'session_id', 'system', 'tld_analysis', 'visualization']
✅ Required artifacts present.
